In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

Download the latest chrome driver from: https://sites.google.com/a/chromium.org/chromedriver/downloads
place it in a directory and take note of the path

In [20]:
browser = webdriver.Chrome("./chromedriver")

In [21]:
browser.get('http://www.instagram.com/')

Perform the following:
1. Log in with instagram credentials. 
2. Search for something.

#### Automated Login

In [22]:
browser.find_element_by_css_selector('p[class="_g9ean"]').find_element_by_tag_name("a").click()

In [23]:
import os

In [24]:
email_field = browser.find_element_by_name("username")

In [25]:
email_field.send_keys(os.environ.get("instagram_email"))

In [26]:
password_field = browser.find_element_by_name("password")

In [27]:
password_field.send_keys(os.environ.get("instagram_password"))

In [28]:
button = browser.find_element_by_css_selector('button[class="_qv64e _gexxb _4tgw8 _njrw0"]')

In [29]:
button.click()

For each post that appears in the search results, extract the caption. Depending on whether the post contains the hashtag "sponsored" take note of the link to the post. 

In [30]:
# inspect source, changes regularly
css_selector = 'div[class="_mck9w _gvoze _tn0ps"]'

what's a div??: https://forums.digitalpoint.com/proxy/V%2FlNRoM4xGeP4jwhh2L%2BjeHwkEChNlXn6jUK3ET1ZceizQ%3D%3D/image.png

**Note**: tag is in purple font, attribute is in yellow font

In [31]:
allposts = browser.find_elements_by_css_selector(css_selector)
list_of_sponsored_posts = []
for post in allposts:
    
    post_image = post.find_element_by_tag_name("img")
    caption = post_image.get_attribute("alt")
    
    post_link = post.find_element_by_tag_name("a")
    link = post_link.get_attribute("href")
    
    if "#sponsored" in caption.lower():
        list_of_sponsored_posts.append(link)
    

In [32]:
list_of_sponsored_posts

['https://www.instagram.com/p/BdtT7R_DkYe/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/BdtwQdHDslx/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdtv4iUBvyZ/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdt5niPjGaU/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdtwp27HwvQ/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/BdtwgA9nU57/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdtv56cFCH7/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/BdtuMpelny1/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/BdtuJJRgnQk/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/BdttRH3FlvX/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdts8vHgbHd/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/BdsCWWdjGe4/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdr_NrdhFgx/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdr7hffBNjm/?tagged=mofsgxstarngage',
 'https://www.instagram.com/p/Bdr7ePwHN2z/?tagge

Visit each post to obtain the instagram username of the influencer

In [33]:
list_of_mof_influencers = []
css_selector = 'div[class="_eeohz"]'
for link in list_of_sponsored_posts:
    browser.get(link)
    time.sleep(1)
    list_of_mof_influencers.append(browser.find_element_by_css_selector(css_selector).text)

In [34]:
list_of_mof_influencers

['awinsomelife',
 'awinsomelife',
 'celinebleh',
 'cheowster',
 'theroycelee',
 'kwleong',
 'mshannahchia',
 'miss_teatime_sg',
 'tjang.fonda',
 'andreatansh',
 'herdailymuse',
 'mandalogy21',
 'kavielteo',
 'denisetaytay',
 'shanelkiehls']

In [35]:
# Remove duplicates (order is lost)
list_of_mof_influencers = list(set(list_of_mof_influencers))

In [36]:
list_of_mof_influencers

['tjang.fonda',
 'kavielteo',
 'andreatansh',
 'denisetaytay',
 'mandalogy21',
 'shanelkiehls',
 'cheowster',
 'theroycelee',
 'kwleong',
 'herdailymuse',
 'miss_teatime_sg',
 'awinsomelife',
 'mshannahchia',
 'celinebleh']

Now that we have the instagram usernames of all the influencers in the campaign, we would like to scrap their individual pages. First, we have to tackle the problem of scrolling.

In [40]:
def scroll_page(n_scrolls):
    body_elem = browser.find_element_by_tag_name("body")
    for i in range(n_scrolls):
        print("scrolling..." + str(i + 1))
        body_elem.send_keys(Keys.END)
        time.sleep(2)
            

In [41]:
scroll_page(5)

scrolling...1
scrolling...2
scrolling...3
scrolling...4
scrolling...5


This block of code saves an image from a given url

In [42]:
import urllib.request

def save_img(img_src, filename):
    urllib.request.urlretrieve(img_src, filename)
    # TODO: Handle http response for errors

From each post, grab the:
1. Link of the post 
2. Img url and Caption

**Edit**: Added influencer column to demonstrate groupby operation

In [57]:
def scrape_links_and_captions_from_page(influencer, css_selector, n_scrolls):
    page_url = "https://www.instagram.com/" + influencer
    browser.get(page_url)
    scroll_page(n_scrolls)
    allposts = browser.find_elements_by_css_selector(css_selector)
    # start with an empty list of data
    data = []
    counter = 0
    print("saving data...")
    for post in allposts:
        post_link = post.find_element_by_tag_name("a")
        post_image = post.find_element_by_tag_name("img")

        link = post_link.get_attribute("href")
        caption = post_image.get_attribute("alt")
        image_url = post_image.get_attribute("src")
        
        # assigns a unique file name to each image
        filename = "./data/%s_%s.jpg"%(influencer,str(counter))
        
        save_img(image_url, filename)
        data.append([influencer,link,caption,filename])
        counter += 1
        
    return data

list indexing: https://i.stack.imgur.com/vIKaD.png

In [58]:
css_selector_posts = 'div[class="_mck9w _gvoze _tn0ps"]'
all_data = []
for influencer in list_of_mof_influencers[-3:]:
    print(influencer)
    data = scrape_links_and_captions_from_page(influencer, css_selector_posts, n_scrolls=5)
    all_data += data

awinsomelife
scrolling...1
scrolling...2
scrolling...3
scrolling...4
scrolling...5
saving data...
mshannahchia
scrolling...1
scrolling...2
scrolling...3
scrolling...4
scrolling...5
saving data...
celinebleh
scrolling...1
scrolling...2
scrolling...3
scrolling...4
scrolling...5
saving data...


### Looking at the data

In [59]:
import pandas as pd
df = pd.DataFrame(all_data, columns=["influencer","links", "captions","filename"])

In [61]:
df["word count"] = df.captions.apply(lambda x: len(x.split(" ")))

In [79]:
df.groupby("influencer").mean()

,word count
influencer,
awinsomelife,39.533333
celinebleh,58.488889
mshannahchia,86.711111


In [52]:
# Save the dataframe to a csv
df.to_csv("mof_ig_data.csv")